# Mixture of 1d Gaussians

In this note, we will solve the mixture of linear regressions problem using the moment method.

This block contains some $\LaTeX$ macros.
$\newcommand{\E}{\mathbb{E}}$
$\renewcommand{\Re}{\mathbb{R}}$
$\newcommand{\oft}[1]{{^{(#1)}}}$
$\newcommand{\oftt}[2]{{^{(#1)#2}}}$

In [3]:
import util
from util import *
import numpy as np
np.set_printoptions(precision=3, suppress=True)
from IPython.display import display, Markdown, Math
from operator import mul
import sympy as sp
sp.init_printing()

## Toy Example

Let's construct a toy example for the rest of this document.

Let $\mu_1, ..., \mu_K \in \Re^D$ be the set of means. Let $\sigma_1, ..., \sigma_N \in \Re^D$ be the set of diagonal covariances.

In [4]:
mus=[-2., 2.]; sigs=[1., np.sqrt(3.)]; pis=[0.5, 0.5]
mu,sigma = sp.symbols('mu,sigma')
display((mu,sigma))
sp.symbols('xi')

In [3]:
def generate_constraints(num_constrs, symbols, mus, sigs, pis):
    K = len(mus)
    H = abs(hermite_coeffs(num_constrs))
    constrs = [0]*num_constrs
    mu = symbols[0]; sig = symbols[1];
    for order in range(num_constrs):
        for i in range(order+1):
            constrs[order] = constrs[order] + H[order,i]* mu**(i) * sig**(order-i)
        constrsval = 0
        for k in range(K):
            constrsval += pis[k]*constrs[order].evalf(subs={mu:mus[k], sig:sigs[k]})
        constrs[order] -= constrsval
    return constrs

def generate_localization_matrices(M, gs):
    #gs = [sig-1, 30-mu**2, 5-sig**2]
    locmatrices = [mm.LocalizingMatrix(M, g) for g in gs]
    Ghs = [lm.get_cvxopt_Gh() for lm in locmatrices]
    return Ghs
    


With this machinery, we can compute the moment polynomials required for the moment method magic!

In [4]:
import MomentMatrix as mm; reload(mm)
from cvxopt import solvers
constrs = generate_constraints(7, [mu,sigma], mus, sigs, pis)
display(constrs)


M = mm.MomentMatrix(3, [mu, sigma], morder='grevlex')
cin = M.get_cvxopt_inputs(constrs)
gs = [sigma-1, 30-mu**2, 5-sigma**2]
Ghs = generate_localization_matrices(M, gs)
Gs=cin['G'] + [Gh['G'] for Gh in Ghs]
hs=cin['h'] + [Gh['h'] for Gh in Ghs]

sol = solvers.sdp(cin['c'],Gs=Gs, \
                  hs=hs, A=cin['A'], b=cin['b'])

M.pretty_print(sol)
print M.extract_solutions_lasserre(sol['x'], Kmax=2)



     pcost       dcost       gap    pres   dres   k/t
 0: -9.3228e+01 -9.3228e+01  2e+04  4e+03  2e+01  1e+00
 1: -9.8397e+03  2.0604e+04  8e+07  1e+05  8e+02  3e+04
 2:  3.7738e+04  3.5151e+05  2e+08  4e+04  3e+02  3e+05
 3:  3.4391e+03  3.1379e+06  2e+08  6e+03  6e+01  3e+06
 4:  3.4960e+03  3.1411e+08  2e+10  6e+03  2e+03  3e+08
 5:  3.5534e+03  3.1420e+10  2e+12  7e+03  2e+05  3e+10
 6:  3.5107e+03  3.0286e+12  2e+14  7e+03  1e+07  3e+12
 7:  3.4943e+03  2.9863e+14  2e+16  6e+03  6e+08  3e+14
 8:  3.3435e+03  2.5955e+16  1e+18  6e+03  1e+11  3e+16
 9:  3.4046e+03  2.5984e+18  2e+20  6e+03  1e+13  3e+18
10:  3.3925e+03  2.5707e+20  2e+22  6e+03  4e+14  3e+20
11:  3.3956e+03  2.5710e+22  2e+24  6e+03  4e+16  3e+22
12:  3.3766e+03  2.5277e+24  1e+26  6e+03  8e+18  3e+24
13:  3.3069e+03  2.3675e+26  1e+28  6e+03  1e+21  2e+26
14:  3.4045e+03  2.3699e+28  1e+30  6e+03  3e+22  2e+28
15:  3.4389e+03  2.3699e+30  1e+32  6e+03  3e+24  2e+30
16:  3.5051e+03  2.3705e+32  2e+34  6e+03  9e+26  

In [95]:
def test_1dmog(mus=[-2., 2.], sigs=[1., np.sqrt(3.)], pis=[0.5, 0.5], deg = 3, degobs=6):
    print 'testing 1d mixture of Gaussians'
    # constraints on parameters here
    
    K = len(mus)
    mu,sig = sp.symbols('mu,sigma')
    M = mm.MomentMatrix(deg, [mu, sig], morder='grevlex')

    num_constrs = degobs+1; # so observe num_constrs-1 moments
    H = abs(hermite_coeffs(num_constrs))
    constrs = [0]*num_constrs
    
    for order in range(num_constrs):
        for i in range(order+1):
            constrs[order] = constrs[order] + H[order,i]* mu**(i) * sig**(order-i)
        constrsval = 0
        for k in range(K):
            constrsval += pis[k]*constrs[order].evalf(subs={mu:mus[k], sig:sigs[k]})
        constrs[order] -= constrsval
    display(constrs)
    cin = M.get_cvxopt_inputs(constrs[1:])

    gs = [sig-1, 30-mu**2, 5-sig**2]
    gs = [sig-1]
    locmatrices = [mm.LocalizingMatrix(M, g) for g in gs]
    Ghs = [lm.get_cvxopt_Gh() for lm in locmatrices]

    Gs=cin['G'] + [Gh['G'] for Gh in Ghs]
    hs=cin['h'] + [Gh['h'] for Gh in Ghs]
    
    sol = solvers.sdp(cin['c'],Gs=Gs, \
                  hs=hs, A=cin['A'], b=cin['b'])
    if 0:
        for i,mono in enumerate(M.matrix_monos):
            trueval = 0;
            if i>0:
                for k in range(K):
                    trueval += pis[k]*mono.evalf(subs={mu:mus[k], sig:sigs[k]})
            else:
                trueval = 1
            print '%s:\t%f\t%f' % (str(mono), sol['x'][i], trueval)
    print M.extract_solutions_lasserre(sol['x'], Kmax=len(mus)+2)
    return M,sol

In [51]:
M_mog,sol_mog=test_1dmog(mus=[-2., 2.], sigs=[1., np.sqrt(3.)], pis=[0.5, 0.5], deg = 3)


testing 1d mixture of Gaussians


the next biggest eigenvalue we are losing is 0.000000
{sigma: [1.7320508129187397, 1.0000000027454958], mu: [1.9999999854161601, -1.9999999851294334]}


In [81]:
Kmog = 2;
pis = np.random.rand(Kmog); pis = pis/sum(pis)
mus = (-5+10*np.random.rand(2)).tolist()
sigs = (1+np.random.rand(Kmog)).tolist()
mus = np.array([-4,4])+(np.random.rand(Kmog)-0.5)
mus = mus.tolist()
from cvxopt import solvers
solvers.options['show_progress'] = False
M_mog,sol_mog=test_1dmog(mus=mus, \
                              sigs=sigs, pis=pis, deg = 3, degobs=6)
print 'Real params:'
display(mus,sigs)


testing 1d mixture of Gaussians


the next biggest eigenvalue we are losing is 0.000003
{sigma: [1.1398207109355758, 1.9256984770269698], mu: [3.5464188747216543, -4.3907186447414928]}
Real params:


In [98]:
Kmog = 3;
pis = np.random.rand(Kmog); pis = pis/sum(pis)
mus = (-5+10*np.random.rand(2)).tolist()
sigs = (1+np.random.rand(Kmog)).tolist()
mus = np.array(np.random.randint(4,size=(Kmog,)))+(np.random.rand(Kmog)-0.5)
#mus = np.array([-4,4])
mus = mus.tolist()
from cvxopt import solvers
solvers.options['show_progress'] = True
solvers.options['maxiters'] = 200
M_mog,sol_mog=test_1dmog(mus=mus, \
                              sigs=sigs, pis=pis, deg = 6, degobs=12)
print 'Real params:'
display(mus,sigs)


testing 1d mixture of Gaussians


     pcost       dcost       gap    pres   dres   k/t
 0:  6.9445e+02  6.9445e+02  2e+04  2e+03  1e+00  1e+00
 1:  1.0473e+03  1.0987e+03  9e+03  8e+02  6e-01  5e+01
 2:  1.0590e+03  1.0681e+03  1e+03  1e+02  1e-01  9e+00
 3:  1.0584e+03  1.0647e+03  5e+02  5e+01  4e-02  6e+00
 4:  1.0705e+03  1.0767e+03  4e+02  2e+01  2e-02  6e+00
 5:  1.2050e+03  1.2343e+03  2e+03  2e+01  2e-02  3e+01
 6:  1.4827e+03  1.5234e+03  2e+03  9e+00  7e-03  4e+01
 7:  1.7826e+03  1.8592e+03  5e+03  9e+00  6e-03  8e+01
 8:  2.1441e+03  2.1924e+03  2e+03  3e+00  2e-03  5e+01
 9:  2.2918e+03  2.3990e+03  7e+03  4e+00  3e-03  1e+02
10:  3.0586e+03  3.1393e+03  4e+03  1e+00  9e-04  8e+01
11:  3.5289e+03  3.6360e+03  6e+03  1e+00  8e-04  1e+02
12:  4.5220e+03  4.6236e+03  4e+03  4e-01  3e-04  1e+02
13:  4.8394e+03  4.9971e+03  8e+03  5e-01  3e-04  2e+02
14:  6.0077e+03  6.1225e+03  5e+03  2e-01  1e-04  1e+02
15:  6.2434e+03  6.3923e+03  8e+03  2e-01  2e-04  1e+02
16:  7.4156e+03  7.4953e+03  3e+03  6e-02  4e-05  